In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score, precision_recall_curve, roc_curve, auc
from sklearn.model_selection import train_test_split 
from scipy.stats import gamma, beta, norm
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support,roc_curve
import statistics
from scipy.optimize import minimize
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score,precision_recall_curve,auc,PrecisionRecallDisplay,average_precision_score
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from statistics import NormalDist
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [2]:
# Read in the same training and set set
df_train=pd.read_csv('/Users/lshi/Desktop/Opioid_Research/Data/StaticSplit_Training_Set.csv')
df_valid=pd.read_csv('/Users/lshi/Desktop/Opioid_Research/Data/StaticSplit_Validation_Set.csv')
df_test=pd.read_csv('/Users/lshi/Desktop/Opioid_Research/Data/StaticSplit_Test_Set.csv')

In [3]:
# All Buyers data
all_buyer = pd.read_csv('/Users/lshi/Desktop/Opioid_Research/Data/Buyers_Info_04_05_2022.csv')

#### Get Buyer business act
df_1 = all_buyer.drop_duplicates(['BUYER_DEA_NO']) 
# Practitioners
df_pr_1 = df_1[(df_1['BUYER_BUS_ACT']=='PRACTITIONER')|(df_1['BUYER_BUS_ACT']=='PRACTITIONER-DW/30')|(df_1['BUYER_BUS_ACT']=='PRACTITIONER-DW/100')|(df_1['BUYER_BUS_ACT']=='PRACTITIONER-DW/275')]
df_pr_1['BUYER_BUS_ACT_d'] = 'Practitioner'
# Retail Pharmacy
df_ret_1 = df_1[(df_1['BUYER_BUS_ACT']=='RETAIL PHARMACY')|(df_1['BUYER_BUS_ACT']=='M/O PHARMACY')]
df_ret_1['BUYER_BUS_ACT_d'] = 'Retail_Pharmacy'
# Chain Pharmacy
df_chain_1 = df_1[(df_1['BUYER_BUS_ACT']=='CHAIN PHARMACY')]
df_chain_1['BUYER_BUS_ACT_d'] = 'Chain_Pharmacy'
# Hospital/Clinics
df_hos_1 = df_1[(df_1['BUYER_BUS_ACT']=='HOSPITAL/CLINIC')|(df_1['BUYER_BUS_ACT']=='CHAIN HOSP/CLINIC')|(df_1['BUYER_BUS_ACT']=='MAINTENANCE')|(df_1['BUYER_BUS_ACT']=='MAINT & DETOX')]
df_hos_1['BUYER_BUS_ACT_d'] = 'Hospital'
# MLP
df_mlp_1 = df_1[(df_1['BUYER_BUS_ACT']=='MLP-NURSE PRACTITIONER')|(df_1['BUYER_BUS_ACT']=='MLP-PHYSICIAN ASSISTANT')|(df_1['BUYER_BUS_ACT']=='MLP-NURSE PRACTITIONER-DW/30')|(df_1['BUYER_BUS_ACT']=='MLP-NURSE PRACTITIONER-DW/100')|(df_1['BUYER_BUS_ACT']=='MLP-PHYSICIAN ASSISTANT-DW/30')|(df_1['BUYER_BUS_ACT']=='MLP-PHYSICIAN ASSISTANT-DW/100')|(df_1['BUYER_BUS_ACT']=='MLP-NATUROPATHIC PHYSICIAN')|(df_1['BUYER_BUS_ACT']=='MLP-OPTOMETRIST')|(df_1['BUYER_BUS_ACT']=='MLP-EUTHANASIA TECHNICIAN')|(df_1['BUYER_BUS_ACT']=='MLP-NURSE PRACTITIONER-DW/100')]
df_mlp_1['BUYER_BUS_ACT_d'] = 'MLP'

all_buyer_1 = pd.concat([df_pr_1, df_ret_1, df_chain_1, df_hos_1, df_mlp_1], axis=0)

In [4]:
all_buyer_1.head()

,BUYER_NAME,BUYER_DEA_NO,BUYER_BUS_ACT,BUYER_STATE,BUYER_COUNTY,BUYER_CITY,BUYER_ZIP,BUYER_ADDRESS1,BUYER_ADDRESS2,BUYER_ADDL_CO_INFO,BUYER_BUS_ACT_d
0,"TABRIZI, HAMID R DMD",BT3484653,PRACTITIONER,MA,MIDDLESEX,MALDEN,2148,"389 MAIN STREET, SUITE 404",NaN,NaN,Practitioner
2,"CIFUENTES, ENRIQUE A. MD",FC1491923,PRACTITIONER,AZ,MARICOPA,TEMPE,85283,2727 W. BASELINE RD. SUITE # 8,NaN,GILA IM & GERIATRICS P.C.,Practitioner
3,"OLSON, MARK C MD",AO6175168,PRACTITIONER,WA,SPOKANE,SPOKANE,99204,601 WEST FIFTH AVE,STE 400,NaN,Practitioner
4,"SLASKI, ANDREW J MD",BS4864903,PRACTITIONER-DW/30,AZ,PIMA,TUCSON,85705,"540 W, PRINCE ROAD",UNIT SE,LA CLINICA DE LA BUENA ESPERANZA,Practitioner
5,"LEVIN, BARBARA A. MD",AM2496811,PRACTITIONER,TN,MONROE,MADISONVILLE,37354,3459 HWY 68,PO BOX 115,NaN,Practitioner


In [5]:
## get retail pharmacy
#df_r_buyer = all_buyer_1[(all_buyer_1['BUYER_BUS_ACT_d']=='Practitioner')|(all_buyer_1['BUYER_BUS_ACT_d']=='MLP')]
df_r_buyer = all_buyer_1[(all_buyer_1['BUYER_BUS_ACT_d']=='Retail_Pharmacy')]

df_train_= df_train[df_train['BUYER_DEA_NO'].isin(df_r_buyer['BUYER_DEA_NO'])]
## get retail pharmacy
df_valid_= df_valid[df_valid['BUYER_DEA_NO'].isin(df_r_buyer['BUYER_DEA_NO'])]
df_test_= df_test[df_test['BUYER_DEA_NO'].isin(df_r_buyer['BUYER_DEA_NO'])]

In [7]:
## Assign df
df_train = df_train_
df_valid = df_valid_
df_test = df_test_

In [8]:
X_vars = ['avg_MME', 'std_MME',
       'median_MME', 'coef_variation', 'max_MME', 'max_med_MME',
       'avg_MME_per_day_c', 'std_MME_per_day_c', 'median_MME_per_day_c',
       'max_MME_per_day_c', 'max_med_MME_per_day_c', 'avg_MME_per_day_per_person',
       'std_MME_per_day_per_person', 'median_MME_per_day_per_person','max_MME_per_day_per_person',
       'max_med_MME_per_day_per_person', 'avg_MME_per_day_per_old_person',
       'std_MME_per_day_per_old_person', 'median_MME_per_day_per_old_person',
       'max_MME_per_day_per_old_person', 'max_med_MME_per_day_per_old_person',
       'Hydrocodone_Share', 'Oxycodone_Share', 'Fentanyl_Share',
       'Morphine_Share', 'Hydro_Oxy_Share', 'Hydro_Oxy_Fen_Share',
       'Hydro_Oxy_Fen_Mor_Share', 'Num_Unique_SKU', 'Num_SKUs_80',
       'Num_Unique_Dist', 'Num_Dist_80', 'Avg_Num_Products',
       'Median_Num_Products', 'Std_Num_Products', 'Max_Num_Products',
       'Avg_Num_Distributors', 'Median_Num_Distributors',
       'Std_Num_Distributors', 'Max_Num_Distributors']

X_train = df_train[X_vars]
Y_train = df_train['bad']

X_valid = df_valid[X_vars]
Y_valid = df_valid['bad']

X_test = df_test[X_vars]
Y_test = df_test['bad']

## Classifier Functions

In [9]:
import scipy.stats as stats
from scipy.stats import gamma, beta, norm
class gamma_classifier():
    
    def __init__(self):
        self.gamma = None
    
    def fit(self, X_train):
        mu = np.mean(X_train)
        var = np.var(X_train)
        shape = mu**2/var
        scale = var/mu
        self.gamma = stats.gamma(shape, scale=scale)
        
    def predict_probs(self, X_test):
        self.probs = 1 - self.gamma.cdf(X_test)
        
        return self.probs
    
    def classify(self, threshold):
        y_pred = np.where(self.probs <= threshold, 1, 0)  ###<=
        
        return y_pred
    
class beta_classifier:
    def fit(self, X_train):
        mu = np.mean(X_train)
        var = np.var(X_train)
        
        v = -1 + (mu * (1 - mu)/var)
        self.beta = beta(a=mu*v, b=(1-mu)*v)
        
    def predict_probs(self, X_test):
        self.probs = 1 - self.beta.cdf(X_test)
        
        return self.probs
    
    def classify(self, threshold):
        y_pred = np.where(self.probs <= threshold, 1, 0)
        
        return y_pred
    
def score_classifier(y_true, y_pred, t=0):
        
    return {"cutoff": t,
            "matrix": confusion_matrix(y_true, y_pred), 
            "f1": f1_score(y_true, y_pred), 
            "precision": precision_score(y_true, y_pred, zero_division=0), 
            "recall": recall_score(y_true, y_pred)}

In [11]:
def score_classifier(y_true, y_pred, t=0):
        
    return {"cutoff": t,
            "matrix": confusion_matrix(y_true, y_pred), 
            "f1": f1_score(y_true, y_pred), 
            "precision": precision_score(y_true, y_pred, zero_division=0), 
            "recall": recall_score(y_true, y_pred)}

## Classification

In [12]:
beta_cols = ["Hydrocodone_Share", "Oxycodone_Share", "Fentanyl_Share", "Morphine_Share", 
             "Hydro_Oxy_Share", "Hydro_Oxy_Fen_Share", "Hydro_Oxy_Fen_Mor_Share"]

In [18]:
### 1 variable
results_1 = np.zeros((X_valid.shape[1],7))
for p in range(len(X_train.columns)):
    feature=X_train.columns[p]
    if feature in ["Median_Num_Distributors"]: continue
    
    train = X_train[feature]
    val = X_valid[feature]
    model = beta_classifier() if feature in beta_cols else gamma_classifier()
    model.fit(train)
    model.predict_probs(val)
    
    ## Crorss-validation
    L = 1000
    cutoff_list=list(np.linspace(0.000001,1,L,endpoint=False))

    f1_list=[]
    precision_list=[]
    recall_list=[]
    for l in range(len(cutoff_list)):
        y_pred = model.classify(cutoff_list[l])
        precision_list.append(precision_recall_fscore_support(Y_valid, y_pred, average='binary')[0])
        recall_list.append(precision_recall_fscore_support(Y_valid, y_pred, average='binary')[1])
        f1_list.append(precision_recall_fscore_support(Y_valid, y_pred, average='binary')[2])

    df_f1 = pd.DataFrame(list(zip(cutoff_list,precision_list,recall_list,f1_list)),
                   columns =['Cutoff', 'Precision','Recall','F1_score'])
    
    cutoff = df_f1.iloc[np.argmax(df_f1['F1_score'])][0]
    
    # Evaluate on test set
    test = X_test[feature]
    model.predict_probs(test)
    y_pred = model.classify(cutoff)
    
    tn, fp, fn, tp = confusion_matrix(Y_test, y_pred).ravel()
    auc_score = roc_auc_score(Y_test, y_pred)
    precision, recall, _ = precision_recall_curve(Y_test, y_pred)
    lift = auc(recall, precision) # recall,precision
    results_1[p,0] = f1_score(Y_test, y_pred, average='binary')
    results_1[p,1] = tp
    results_1[p,2] = fp
    results_1[p,3] = fn
    results_1[p,4] = tn
    results_1[p,5] = np.max(df_f1['F1_score'])# f1 in validation set
    results_1[p,6] = cutoff
    
results_1 = pd.DataFrame(results_1) 
results_1

,0,1,2,3,4,5,6
0,0.384615,5.0,0.0,16.0,1850.0,0.250000,0.034001
1,0.039514,13.0,624.0,8.0,1226.0,0.042042,0.433001
2,0.600000,9.0,0.0,12.0,1850.0,0.500000,0.031001
3,0.011019,2.0,340.0,19.0,1510.0,0.034384,0.181001
4,0.032103,10.0,592.0,11.0,1258.0,0.034056,0.453001
5,0.036641,12.0,622.0,9.0,1228.0,0.032641,0.494001
6,0.500000,7.0,0.0,14.0,1850.0,0.444444,0.036001
7,0.039761,10.0,472.0,11.0,1378.0,0.042226,0.299001
8,0.727273,12.0,0.0,9.0,1850.0,0.500000,0.033001
9,0.500000,7.0,0.0,14.0,1850.0,0.384615,0.042001


In [19]:
### 2 variables 
## fixing cutoff at the best for all variables
results_1 = pd.DataFrame(results_1)
cutoff_star_1 = results_1.iloc[np.argmax(results_1.iloc[:,0]),-1]
best_1 = np.argmax(results_1.iloc[:,0])
model1 = beta_classifier() if X_train.columns[best_1] in beta_cols else gamma_classifier()
model1.fit(X_train.iloc[:,best_1])
model1.predict_probs(X_valid.iloc[:,best_1])
best_1_pred = model1.classify(cutoff_star_1)

results_2 = np.zeros((X_valid.shape[1],8))

for p in range(len(X_train.columns)):
    if p==best_1:#var with best f1 from model 1
        pass
    else:
        feature=X_train.columns[p]
        if feature in ["Median_Num_Distributors"]: continue

        cutoff_list1 = [cutoff_star_1]#fix cutoff at the best cutoff

        cutoff_star = results_1.iloc[p,-1]
        cutoff_list2 = [cutoff_star] # Use cutoff from validation set trained from 1 variable model
       
        #Predict on Test using the best cutoffs
        cutoff1 = cutoff_list1[0]
        cutoff2 = cutoff_list2[0]
        model1.predict_probs(X_test.iloc[:,best_1])
        model.fit(X_train[feature])
        model.predict_probs(X_test[feature])

        Y_pred=model1.classify(cutoff_star_1)|model.classify(cutoff2)
        #df_valid['Pred_clean']=np.where((pr.iloc[:,p]<cutoff2), 0, 1)
        #Y_valid = np.where(df_valid['clean']==0, 1, 0)
        
        tn, fp, fn, tp = confusion_matrix(Y_test, Y_pred).ravel()
        auc_score = roc_auc_score(Y_test, Y_pred)
        precision, recall, _ = precision_recall_curve(Y_test, Y_pred)
        lift = auc(recall, precision) # recall,precision

        results_2[p,0] = precision_recall_fscore_support(Y_test,Y_pred, average='binary')[2] #f1 store
        tn, fp, fn, tp = confusion_matrix(Y_test, Y_pred).ravel()
        results_2[p,1] = tp
        results_2[p,2] = fp
        results_2[p,3] = fn
        results_2[p,4] = tn
        results_2[p,5] = np.max(f1_list) # f1 in validation set
        results_2[p,6] = cutoff1
        results_2[p,7] = cutoff2
results_2 = pd.DataFrame(results_2)
results_2

,0,1,2,3,4,5,6,7
0,0.727273,12.0,0.0,9.0,1850.0,0.5,0.033001,0.034001
1,0.048411,16.0,624.0,5.0,1226.0,0.5,0.033001,0.433001
2,0.727273,12.0,0.0,9.0,1850.0,0.5,0.033001,0.031001
3,0.069519,13.0,340.0,8.0,1510.0,0.5,0.033001,0.181001
4,0.044657,14.0,592.0,7.0,1258.0,0.5,0.033001,0.453001
5,0.045593,15.0,622.0,6.0,1228.0,0.5,0.033001,0.494001
6,0.727273,12.0,0.0,9.0,1850.0,0.5,0.033001,0.036001
7,0.055227,14.0,472.0,7.0,1378.0,0.5,0.033001,0.299001
8,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
9,0.727273,12.0,0.0,9.0,1850.0,0.5,0.033001,0.042001


In [20]:
### 3 variables ####
## fixing cutoff at the best for all variables
results_1 = pd.DataFrame(results_1)
cutoff_star_1 = results_1.iloc[np.argmax(results_1.iloc[:,0]),-1]
best_1 = np.argmax(results_1.iloc[:,0])

cutoff_star_2 = results_2.iloc[np.argmax(results_2.iloc[:,0]),-1]
best_2 = np.argmax(results_2.iloc[:,0])

model1 = beta_classifier() if X_train.columns[best_1] in beta_cols else gamma_classifier()
model1.fit(X_train.iloc[:,best_1])

model2 = beta_classifier() if X_train.columns[best_2] in beta_cols else gamma_classifier()
model2.fit(X_train.iloc[:,best_2])

results_3 = np.zeros((X_valid.shape[1],9))

for p in range(len(X_train.columns)):
    if p==best_1 or p==best_2:#var with best f1 from model 1
        pass
    else:
        feature=X_train.columns[p]
        if feature in ["Median_Num_Distributors"]: continue

        cutoff_star_3 = results_2.iloc[p,-1] # Use cutoff from validation set trained from 1 variable model
       
        #Predict on Test using the best cutoffs
        model1.predict_probs(X_test.iloc[:,best_1])
        model2.predict_probs(X_test.iloc[:,best_2])
        
        model.fit(X_train[feature])
        model.predict_probs(X_test[feature])

        Y_pred=model1.classify(cutoff_star_1)|model2.classify(cutoff_star_2)|model.classify(cutoff_star_3)
        
        tn, fp, fn, tp = confusion_matrix(Y_test, Y_pred).ravel()
        auc_score = roc_auc_score(Y_test, Y_pred)
        precision, recall, _ = precision_recall_curve(Y_test, Y_pred)
        lift = auc(recall, precision) # recall,precision

        results_3[p,0] = precision_recall_fscore_support(Y_test,Y_pred, average='binary')[2] #f1 store
        tn, fp, fn, tp = confusion_matrix(Y_test, Y_pred).ravel()
        results_3[p,1] = tp
        results_3[p,2] = fp
        results_3[p,3] = fn
        results_3[p,4] = tn
        results_3[p,5] = np.max(f1_list) # f1 in validation set
        results_3[p,6] = cutoff_star_1
        results_3[p,7] = cutoff_star_2
        results_3[p,8] = cutoff_star_3
results_3 = pd.DataFrame(results_3)
results_3

,0,1,2,3,4,5,6,7,8
0,0.864865,16.0,0.0,5.0,1850.0,0.5,0.033001,0.011001,0.034001
1,0.054299,18.0,624.0,3.0,1226.0,0.5,0.033001,0.011001,0.433001
2,0.864865,16.0,0.0,5.0,1850.0,0.5,0.033001,0.011001,0.031001
3,0.084881,16.0,340.0,5.0,1510.0,0.5,0.033001,0.011001,0.181001
4,0.050874,16.0,592.0,5.0,1258.0,0.5,0.033001,0.011001,0.453001
5,0.051515,17.0,622.0,4.0,1228.0,0.5,0.033001,0.011001,0.494001
6,0.864865,16.0,0.0,5.0,1850.0,0.5,0.033001,0.011001,0.036001
7,0.066667,17.0,472.0,4.0,1378.0,0.5,0.033001,0.011001,0.299001
8,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
9,0.864865,16.0,0.0,5.0,1850.0,0.5,0.033001,0.011001,0.042001


In [21]:
### 4 variables ####
## fixing cutoff at the best for all variables
results_1 = pd.DataFrame(results_1)
cutoff_star_1 = results_1.iloc[np.argmax(results_1.iloc[:,0]),-1]
best_1 = np.argmax(results_1.iloc[:,0])
model1 = beta_classifier() if X_train.columns[best_1] in beta_cols else gamma_classifier()
model1.fit(X_train.iloc[:,best_1])

cutoff_star_2 = results_2.iloc[np.argmax(results_2.iloc[:,0]),-1]
best_2 = np.argmax(results_2.iloc[:,0])
model2 = beta_classifier() if X_train.columns[best_2] in beta_cols else gamma_classifier()
model2.fit(X_train.iloc[:,best_2])

cutoff_star_3 = results_3.iloc[np.argmax(results_3.iloc[:,0]),-1]
best_3 = np.argmax(results_3.iloc[:,0])
model3 = beta_classifier() if X_train.columns[best_3] in beta_cols else gamma_classifier()
model3.fit(X_train.iloc[:,best_3])

results_4 = np.zeros((X_valid.shape[1],10))

for p in range(len(X_train.columns)):
    if p==best_1 or p==best_2 or p==best_3:#var with best f1 from model 1
        pass
    else:
        feature=X_train.columns[p]
        if feature in ["Median_Num_Distributors"]: continue

        cutoff_star_4 = results_3.iloc[p,-1] # Use cutoff from validation set trained from 1 variable model
       
        #Predict on Test using the best cutoffs
        model1.predict_probs(X_test.iloc[:,best_1])
        model2.predict_probs(X_test.iloc[:,best_2])
        model3.predict_probs(X_test.iloc[:,best_3])
        
        model.fit(X_train[feature])
        model.predict_probs(X_test[feature])

        Y_pred=model1.classify(cutoff_star_1)|model2.classify(cutoff_star_2)|model3.classify(cutoff_star_3)|model.classify(cutoff_star_4)
        #df_valid['Pred_clean']=np.where((pr.iloc[:,p]<cutoff2), 0, 1)
        #Y_valid = np.where(df_valid['clean']==0, 1, 0)
        
        tn, fp, fn, tp = confusion_matrix(Y_test, Y_pred).ravel()
        auc_score = roc_auc_score(Y_test, Y_pred)
        precision, recall, _ = precision_recall_curve(Y_test, Y_pred)
        lift = auc(recall, precision) # recall,precision

        results_4[p,0] = precision_recall_fscore_support(Y_test,Y_pred, average='binary')[2] #f1 store
        tn, fp, fn, tp = confusion_matrix(Y_test, Y_pred).ravel()
        results_4[p,1] = tp
        results_4[p,2] = fp
        results_4[p,3] = fn
        results_4[p,4] = tn
        results_4[p,5] = np.max(f1_list) # f1 in validation set
        results_4[p,6] = cutoff_star_1
        results_4[p,7] = cutoff_star_2
        results_4[p,8] = cutoff_star_3
        results_4[p,9] = cutoff_star_4
results_4 = pd.DataFrame(results_4)
results_4

,0,1,2,3,4,5,6,7,8,9
0,0.950000,19.0,0.0,2.0,1850.0,0.5,0.033001,0.011001,0.043001,0.034001
1,0.060150,20.0,624.0,1.0,1226.0,0.5,0.033001,0.011001,0.043001,0.433001
2,0.950000,19.0,0.0,2.0,1850.0,0.5,0.033001,0.011001,0.043001,0.031001
3,0.100000,19.0,340.0,2.0,1510.0,0.5,0.033001,0.011001,0.043001,0.181001
4,0.060127,19.0,592.0,2.0,1258.0,0.5,0.033001,0.011001,0.043001,0.453001
5,0.060332,20.0,622.0,1.0,1228.0,0.5,0.033001,0.011001,0.043001,0.494001
6,0.950000,19.0,0.0,2.0,1850.0,0.5,0.033001,0.011001,0.043001,0.036001
7,0.077973,20.0,472.0,1.0,1378.0,0.5,0.033001,0.011001,0.043001,0.299001
8,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
9,0.950000,19.0,0.0,2.0,1850.0,0.5,0.033001,0.011001,0.043001,0.042001
